In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

In [2]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten
from keras.preprocessing.image import ImageDataGenerator

In [3]:

train = ImageDataGenerator(rescale=1/255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
validation = ImageDataGenerator(rescale=1/255)

In [4]:

train_image_gen = train.flow_from_directory('C:/Users/mayan/Desktop/realtime driver drowsiness detection Project/dataset/train/',target_size=(100,100),batch_size=32,color_mode='grayscale',class_mode='binary')
valid_image_gen = validation.flow_from_directory('C:/Users/mayan/Desktop/realtime driver drowsiness detection Project/dataset/test/',target_size=(100,100),batch_size=32,color_mode='grayscale',class_mode='binary')


Found 1234 images belonging to 2 classes.
Found 218 images belonging to 2 classes.


In [5]:
img_size = 224

In [6]:
batch_size=32
SPE= len(train_image_gen.classes)//batch_size 
VS = len(valid_image_gen.classes)//batch_size 
print(SPE,VS)

print(train_image_gen.class_indices)

model = Sequential()
model.add(Conv2D(64,(3,3),input_shape=(100,100,1),activation="relu"))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(32,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(32,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(16,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(16,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(128,activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(64,activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

results = model.fit_generator(train_image_gen,validation_data=valid_image_gen,epochs=10,steps_per_epoch=SPE,validation_steps=VS)

model.save("CNN__model.h5")

38 6
{'Closed': 0, 'Open': 1}


C:\Users\mayan\AppData\Local\Temp\ipykernel_20264\3841714561.py:37: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  results = model.fit_generator(train_image_gen,validation_data=valid_image_gen,epochs=10,steps_per_epoch=SPE,validation_steps=VS)


Epoch 1/10
38/38 [==============================] - 23s 576ms/step - loss: 0.6873 - accuracy: 0.5216 - val_loss: 0.6497 - val_accuracy: 0.5365
Epoch 2/10
38/38 [==============================] - 15s 400ms/step - loss: 0.5249 - accuracy: 0.7604 - val_loss: 0.2741 - val_accuracy: 0.9167
Epoch 3/10
38/38 [==============================] - 15s 396ms/step - loss: 0.3424 - accuracy: 0.8769 - val_loss: 0.2314 - val_accuracy: 0.9271
Epoch 4/10
38/38 [==============================] - 16s 410ms/step - loss: 0.2710 - accuracy: 0.9043 - val_loss: 0.1408 - val_accuracy: 0.9479
Epoch 5/10
38/38 [==============================] - 15s 383ms/step - loss: 0.1986 - accuracy: 0.9201 - val_loss: 0.1869 - val_accuracy: 0.9323
Epoch 6/10
38/38 [==============================] - 14s 354ms/step - loss: 0.1929 - accuracy: 0.9309 - val_loss: 0.1448 - val_accuracy: 0.9479
Epoch 7/10
38/38 [==============================] - 14s 356ms/step - loss: 0.1579 - accuracy: 0.9434 - val_loss: 0.1415 - val_accuracy: 0.9375

In [7]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

pygame 2.1.2 (SDL 2.0.18, Python 3.10.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [8]:

mixer.init()
sound = mixer.Sound('alarm.wav')

In [9]:

face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
leye = cv2.CascadeClassifier('haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('haarcascade_righteye_2splits.xml')
eyes=cv2.CascadeClassifier('haarcascade_eye.xml')

In [10]:
face

<CascadeClassifier 000001431FDB3A10>

In [11]:
leye

<CascadeClassifier 000001436FAB33F0>

In [12]:
reye

<CascadeClassifier 000001431FDB39B0>

In [13]:
eyes

<CascadeClassifier 000001431FDB3930>

In [14]:
lbl = "Open"
model = load_model('CNN__model.h5')
path = os.getcwd()
cap = cv2.VideoCapture(0) #to access the camera 
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]


In [ ]:


while(True):
    ret, frame = cap.read()   
    height,width = frame.shape[:2] 

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25)) #Face detection
    eye=eyes.detectMultiScale(gray) #eye detection
    left_eye = leye.detectMultiScale(gray) #Left eye detection
    right_eye =  reye.detectMultiScale(gray) #Right eye detection
    
    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) ,thickness=cv2.FILLED)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (150,150,150) , 1) 
        
    for (x,y,w,h) in eye:
        cv2.rectangle(frame, (x,y),(x+w,y+h) ,(150,150,150) , 1) 
       
    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(100,100)) 
        r_eye= r_eye/255                  
        r_eye=  r_eye.reshape(100,100,-1)
        r_eye = np.expand_dims(r_eye,axis=0)
        rpred = model.predict(r_eye)[0]

        if(rpred[0]>0.5):
            lbl='Open' 
        else:
            lbl='Closed'
        break

    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)  
        l_eye = cv2.resize(l_eye,(100,100))
        l_eye= l_eye/255
        l_eye=l_eye.reshape(100,100,-1)
        l_eye = np.expand_dims(l_eye,axis=0)
        lpred = model.predict(l_eye)[0]
        if(lpred[0]>0.5):
            lbl='Open'   
        else:
            lbl='Closed'
        break

    cv2.putText(frame,lbl,(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)

    if lbl=="Open":score = 0
    else: score -= 1
    
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
    if score <= -10:
        #person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        
        
        if lbl == 'Open': 
            score = 0
            mixer.quit()
        else:
            sound.play()
            cv2.putText(frame,'Wake up !!',(100,height-50), font, 1,(255,255,255),1,cv2.LINE_AA)

        
        if(thicc<16):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc) 

    cv2.imshow('Driver drowsiness detection',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:

cap.release()
cv2.destroyAllWindows()